In [ ]:
from urllib.request import urlretrieve
import os
import numpy as np
import tensorflow as tf
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cv2
from scipy import signal
from numpy import linalg as LA
from scipy import ndimage
import random
from matplotlib.patches import Rectangle

def load_img(img_path):
    # read img and fft 
    img   = cv2.imread(img_path,0)
    img   = cv2.resize(img, (256, 256), interpolation=cv2.INTER_CUBIC)
    f_1   = np.fft.fft2(img)
    fshift_1 = np.fft.fftshift(f_1)
    magnitude_spectrum_1  = np.log(np.abs(fshift_1))
    
    magnitude_spectrum_1 = magnitude_spectrum_1.copy()
    magnitude_spectrum_1  = cv2.normalize(magnitude_spectrum_1,  magnitude_spectrum_1, 0, 255, cv2.NORM_MINMAX)
    #ret,magnitude_spectrum_1 = cv2.threshold(magnitude_spectrum_1,150,255,cv2.THRESH_BINARY) #--------------------------------------- 解掉註解能夠二值化
#     4-6沒有二值化 7-9有二值化(127) 10-12(200) 13-15(150)
    #plt.imshow(magnitude_spectrum_1,cmap='gray') #--------------------------------------- 解掉註解能夠看最後一張load 進來的圖長怎樣
    return magnitude_spectrum_1

def load_data():
    imgs   = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    ra_info = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    ra_dataframe  = pd.read_csv('./rough_data.csv', header=None)
    
    for k in imgs.keys():
        dir = './' + k + '/'
        for file in os.listdir(dir):
            
            if not file.lower().endswith('.jpg'):
                continue
            try:
                num,right  = file.split("_",1)
                dire,right = file.split(".",1)
                row        = 0
                col        = 0
                if dire == 'left':
                    col = 0
                else:
                    col = 1
                row = int(num) - 1
                ra_value = ra_dataframe.iloc[row,col]
                resized_img = load_img(os.path.join(dir, file))
            except OSError:
                continue
            imgs[k].append(resized_img)
            ra_info[k].append(ra_value)
    return ra_info,imgs        

def conv_img(img,filter_design):
    # apply convolve2d , return conv_array and std axis
    conv_array = signal.convolve2d(img, filter_design , mode='valid')
    x_axis_array,y_axis_array = np.where(conv_array > (np.mean(conv_array)+(np.max(conv_array)-np.mean(conv_array))/2))
    #(center_x,center_y) = ndimage.measurements.center_of_mass(conv_array)
    return conv_array,[np.std(x_axis_array),np.std(y_axis_array)]

def conv_iter(img_dic,filt):
    result_conv = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    axis  = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    for k in img_dic.keys():
        for img in img_dic[k]:
            conv_array,axis_value = conv_img(img ,filt)
            result_conv[k].append(conv_array)
            axis[k].append(axis_value)
    return result_conv,axis

def oppent_display(axis):
    opponent_sum = 0
    # rough1.0-1.5
    mean_x = np.mean(np.array(axis['rough1.0-1.5'])[:,0])
    mean_y = np.mean(np.array(axis['rough1.0-1.5'])[:,1])
    # * 1 std
    width  = np.std(np.array(axis['rough1.0-1.5'])[:,0])*2
    height = np.std(np.array(axis['rough1.0-1.5'])[:,1])*2
    left_x = mean_x - width/2
    bottom_y = mean_y - height/2
    ac     = Rectangle((left_x, bottom_y), width, height,facecolor = 'None')
    for element in np.array(axis['rough1.5-2.5']):
        if ac.contains_point(element):
            opponent_sum += 1

    
    # rough1.5-2.5
    mean_x = np.mean(np.array(axis['rough1.5-2.5'])[:,0])
    mean_y = np.mean(np.array(axis['rough1.5-2.5'])[:,1])
    # * 1 std
    width  = np.std(np.array(axis['rough1.5-2.5'])[:,0])*2
    height = np.std(np.array(axis['rough1.5-2.5'])[:,1])*2
    left_x = mean_x - width/2
    bottom_y = mean_y - height/2
    ac     = Rectangle((left_x, bottom_y), width, height,facecolor = 'None')
    for element in np.array(axis['rough1.0-1.5']):
        if ac.contains_point(element):
            opponent_sum += 1

    return opponent_sum
    
if __name__ == '__main__':
    color   = {'rough1.0-1.5': 'b' , 'rough1.5-2.5': 'g'}
    plt_store_x = {'rough1.0-1.5': [] , 'rough1.5-2.5': []} 
    plt_store_y = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    plt_store_dis = []
    
    best             = 60
    best_filter = [[0,0,0],[0,0,0],[0,0,0]]
    best_iter   = 0
    
    test_list = []
    # try iterate
    iteration_try = 100    #-------------------- 此處為跑 100 個 隨機 filter
    # define iterate number
    iteration_num = 110    #-------------------- 每個 filter 跑 127次卷積 (因為效果問題 降為110次)
    
    # to store best
    best= 60
    best_filter = [[0,0,0],[0,0,0],[0,0,0]]
    best_iter   = 0
    
    # load image and return spectrum
    ra_info,org_dic = load_data()
    time = 0
    
    # --------------- 下面為產生 1 ~ 511 的 binary number array
    for i in range(1,512,1):
        number = "{0:09b}".format(i)
        test_filter = [[int(number[0]),int(number[1]),int(number[2])],
                       [int(number[3]),int(number[4]),int(number[5])],
                       [int(number[6]),int(number[7]),int(number[8])]]
        test_list.append(test_filter)
    
    # -------------------------------------------------------------------------------
    
    for i in range(len(test_list)):
        # --------------- 下面為隨機產生 0 , 1 的 filter random.randint(0,1) 是產生 0 到 1 的隨機整數
#         test_filter = [[random.randint(0,1),random.randint(0,1),random.randint(0,1)],
#                        [random.randint(0,1),random.randint(0,1),random.randint(0,1)],
#                        [random.randint(0,1),random.randint(0,1),random.randint(0,1)]]
        test_filter = test_list[i]
        img_dic = org_dic
        if time % 10 == 0:
            print('iterate round: ',time)
        time += 1
        for j in range(iteration_num):
            img_dic,axis = conv_iter(img_dic,test_filter)
            oppen = oppent_display(axis)
            if oppen < best:
                best = oppen
                best_filter = test_filter
                best_iter = j
                print('now best_filter: ',best_filter)
                print('now best_oppent: ',best) 
                print('now best_conv: ',best_iter)
                print('-----------------------')
    print('end!')